# Objectives

- [x] get data from database
- [ ] data wrangling
- [ ] ml model 
- [ ] compare with traditional prediction

# source

 data provided by Openweathermap

# Imports

In [11]:
# os
import os

#googlesheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

#data wrangling
import pandas as pd
import numpy as np

#data science
import calmap

#data visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# Data imports

## google sheet authentication

In [2]:
#authentication to google api using JSON credential file
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('api_data/WDcred.json', scope)
client = gspread.authorize(creds)

## Historical weather 

In [3]:
#get historical weather berlin sheet
hist_weather = client.open('current_weather_berlin').sheet1

data = hist_weather.get_all_values()
headers = data.pop(0)

df_hist = pd.DataFrame(data, columns=headers)

In [7]:
df_hist.tail()

datetime temperature humidity pressure windspeed winddir  \
331  2020-04-02 05:38:00        3.96       75     1013       3.1     250   
332  2020-04-02 06:38:17        4.08       75     1013       4.6     240   
333  2020-04-02 07:44:16        4.38       75     1012       4.6     240   
334  2020-04-02 08:45:27        4.45       80     1012       4.6     260   
335  2020-04-02 16:47:04       11.27       39     1008       8.2     260   

    cloudcoverage weather    description weathericon sunrisetime sunsettime  
331            61  Clouds  broken clouds         04n    06:37:27   19:41:52  
332            80  Clouds  broken clouds         04d    06:37:27   19:41:52  
333            80  Clouds  broken clouds         04d    06:37:27   19:41:52  
334            80  Clouds  broken clouds         04d    06:37:27   19:41:52  
335            76  Clouds  broken clouds         04d    06:37:27   19:41:52

## get Openweathermap forecast 

In [5]:
#get forecast weather berlin sheet from open weather map
#forecast_weather = client.open('forecast_weather_berlin')
forecast_weather = client.open('forecast_weather_berlin').sheet1

data = forecast_weather.get_all_values()
headers = data.pop(0)

df_fore = pd.DataFrame(data, columns=headers)

In [6]:
df_fore.head()

temp_f humidity_f pressure_f wind_speed_f wind_dir_f  \
0  2020-03-31 12:00:00   6.17         61       1030         2.09        347   
1  2020-03-31 15:00:00   6.36         60       1029         2.83        315   
2  2020-03-31 18:00:00    3.3         75       1028         2.27        302   
3  2020-03-31 21:00:00   1.56         82       1028          1.9        258   
4  2020-04-01 00:00:00   0.25         86       1026         3.21        245   

  cloudcoverage_f weather_f    description_f icon_f  
0              51    Clouds    broken clouds    04d  
1             100    Clouds  overcast clouds    04d  
2              89    Clouds  overcast clouds    04n  
3               0     Clear        clear sky    01n  
4               0     Clear        clear sky    01n

# Wrangle

## column rename 

In [25]:
df_fore.rename(columns={
    '':'datetime'
    "temp_f": "temperature",
    "humidity_f": "temperature",
    "pressure_f": "temperature",
    "wind_speed_f": "temperature",
    "wind_dir_f": "temperature"},inplace=True)
df_fore.columns

Index(['', 'a', 'humidity_f', 'pressure_f', 'wind_speed_f', 'wind_dir_f',
       'cloudcoverage_f', 'weather_f', 'description_f', 'icon_f'],
      dtype='object')

## join df

In [19]:
combo=[df_hist,df_fore]

## drop icon

In [ ]:
for df in combo:
    df.drop('weathericon',axis =1, inplace=True)

In [15]:
df_hist[['temperature','humidity','pressure','windspeed','cloudcoverage']].astype(float)

temperature  humidity  pressure  windspeed  cloudcoverage
0          13.91      71.0    1025.0        4.1           75.0
1          13.81      67.0    1025.0        3.6           75.0
2          13.48      71.0    1025.0        3.6           75.0
3          13.65      67.0    1025.0        3.1           40.0
4          13.26      71.0    1025.0        3.6           40.0
..           ...       ...       ...        ...            ...
331         3.96      75.0    1013.0        3.1           61.0
332         4.08      75.0    1013.0        4.6           80.0
333         4.38      75.0    1012.0        4.6           80.0
334         4.45      80.0    1012.0        4.6           80.0
335        11.27      39.0    1008.0        8.2           76.0

[336 rows x 5 columns]

In [16]:
df_hist.head()

datetime temperature humidity pressure windspeed winddir  \
0  2020-03-19 11:45:50       13.91       71     1025       4.1     290   
1  2020-03-19 12:50:48       13.81       67     1025       3.6     330   
2  2020-03-19 13:46:46       13.48       71     1025       3.6     350   
3  2020-03-19 14:52:21       13.65       67     1025       3.1     360   
4  2020-03-19 15:52:40       13.26       71     1025       3.6     340   

  cloudcoverage weather       description sunrisetime sunsettime  
0            75  Clouds     broken clouds    06:10:36   18:17:28  
1            75  Clouds     broken clouds    06:10:36   18:17:28  
2            75  Clouds     broken clouds    06:10:36   18:17:28  
3            40  Clouds  scattered clouds    06:10:36   18:17:28  
4            40  Clouds  scattered clouds    06:10:36   18:17:28

In [17]:
df_hist.describe(include='all')

datetime temperature humidity pressure windspeed winddir  \
count                   336         336      336      336       336     336   
unique                  335         300       71       31        15      28   
top     2020-03-31 13:29:30        4.94       47     1025       3.1      50   
freq                      2           3       11       34        63      27   

       cloudcoverage weather description sunrisetime sunsettime  
count            336     336         336         336        336  
unique            38       5           9          15         15  
top                0   Clear   clear sky    06:42:09   19:38:23  
freq             146     168         168          27         27

In [18]:
df_hist.temperature.iloc[1]

'13.81'

In [ ]:
df_hist.drop('weathericon',axis =1, inplace=True)
df_hist.head()